In [2]:
import json
import re
from rich import print

In [3]:
def decode_dict(data):
    if isinstance(data, dict):
        decoded_data = {}
        for key, value in data.items():
            if key == "Contenido":
                decoded_data[key] = decode_dict(value)
            elif key not in ["TF", "TF_Etiquetado"]:
                decoded_data[key] = decode_dict(value)
        return decoded_data
    elif isinstance(data, list):
        return [decode_dict(item) for item in data]
    elif isinstance(data, str):
        try:
            return data.encode("latin_1").decode("utf-8")
        except UnicodeDecodeError:
            return data.encode("latin_1").decode("utf-8", errors="replace")
    else:
        return data

In [4]:
def parse_text(text):
    # Remove leading/trailing whitespace and newline characters
    text = text.strip()

    # Regular expression patterns
    codigo_pattern = r'CODIGO\s+DE\s+COMERCIO'
    libro_pattern = r'LIBRO\s+([A-Z]+)'

    # Initialize the result dictionary
    result = {}

    # Extract CODIGO DE COMERCIO
    codigo_match = re.search(codigo_pattern, text)
    if codigo_match:
        codigo = codigo_match.group()
        result[codigo] = {}

        remaining_text = text[codigo_match.end():]

        # Extract LIBROs
        libro_matches = list(re.finditer(libro_pattern, remaining_text))
        num_libros = len(libro_matches)

        for i in range(num_libros):
            libro_match = libro_matches[i]
            libro = f'LIBRO {libro_match.group(1)}'

            if i < num_libros - 1:
                next_libro_match = libro_matches[i + 1]
                libro_text = remaining_text[libro_match.end():next_libro_match.start()].strip()
            else:
                libro_text = remaining_text[libro_match.end():].strip()

            result[codigo][libro] = libro_text

    return result

In [5]:
def parse_libros(codigo_data):
    # Dictionary of desired words for TITULO sections
    titulo_words = {
        "TITULO PRELIMINAR": "TITULO PRELIMINAR",
        "TITULO PRIMERO": "TITULO PRIMERO",
        "TITULO SEGUNDO": "TITULO SEGUNDO",
        "TITULO TERCERO": "TITULO TERCERO",
        "TITULO CUARTO": "TITULO CUARTO",
        "TITULO QUINTO": "TITULO QUINTO",
        "TITULO SEXTO": "TITULO SEXTO",
        "TITULO SEPTIMO": "TITULO SEPTIMO",
        "TITULO OCTAVO": "TITULO OCTAVO",
        "TITULO NOVENO": "TITULO NOVENO",
        "TITULO DECIMO": "TITULO DECIMO",
        "TITULO DECIMO PRIMERO": "TITULO DECIMO PRIMERO"
    }

    # Regular expression pattern for TITULO sections
    titulo_pattern = r'\\n\\n\s*(' + '|'.join(map(re.escape, titulo_words.keys())) + r')\s*\\n'

    # Iterate over each LIBRO key
    for libro_key in codigo_data.keys():
        libro_text = codigo_data[libro_key]

        # Check if libro_text is a string
        if isinstance(libro_text, str):
            # Extract TITULOs
            titulo_matches = list(re.finditer(titulo_pattern, libro_text, re.DOTALL))
            num_titulos = len(titulo_matches)

            # Create a new dictionary to store the TITULOs for the current LIBRO
            titulo_dict = {}

            # Extract the content before the first TITULO
            if titulo_matches:
                first_titulo_start = titulo_matches[0].start()
                before_first_titulo = libro_text[:first_titulo_start].strip()
                if before_first_titulo:
                    titulo_dict["CONTENIDO_ANTES_DE_TITULOS"] = before_first_titulo

            for i in range(num_titulos):
                titulo_match = titulo_matches[i]
                titulo = titulo_match.group(1).strip()

                if i < num_titulos - 1:
                    next_titulo_match = titulo_matches[i + 1]
                    titulo_text = libro_text[titulo_match.end():next_titulo_match.start()].strip()
                else:
                    titulo_text = libro_text[titulo_match.end():].strip()

                titulo_dict[titulo] = titulo_text

            # Update the LIBRO key with the parsed TITULO dictionary
            codigo_data[libro_key] = titulo_dict
        else:
            # If libro_text is not a string, skip processing it
            continue

    return codigo_data

In [6]:
def print_dict_structure(data, indent=0):
    for key in data:
        print(' ' * indent + key)
        if isinstance(data[key], dict):
            print_dict_structure(data[key], indent + 2)

In [7]:
path = "C:\\Users\\ferna\\.vscode\\GitHub\\MexicanLaws_PrePro_DataSet\\MexicanLaws_PrePro_DataSet\\CÓDIGO DE COMERCIO"
filename = "CÓDIGO DE COMERCIO.json"

In [8]:
with open(f"{path}\\{filename}", "r", encoding="latin_1") as file:
    data = json.load(file)

In [9]:
# Recursively decode the text values
decoded_data = decode_dict(data)

In [10]:
new_dict = {
    'Nombre': data['Nombre'],
    'Títulos': data['Títulos']['Transitorio']['Contenido'],
    'TEXTO VIGENTE': data['TEXTO VIGENTE']['Contenido']
}

# Recursively decode the text values and filter the desired keys
decoded_data = decode_dict(new_dict)

In [11]:
print(decoded_data['TEXTO VIGENTE'][0:-1])

Última reforma publicada DOF 28-03-2018 

 
Cantidades actualizadas por Acuerdo DOF 31-12-2018 

 
 
 
El Presidente de la República se ha servido dirigirme el decreto que sigue: 

 
"PORFIRIO DIAZ, Presidente Constitucional de los Estados Unidos Mexicanos, á sus habitantes, 

sabed: 
 
Que en virtud de la autorización concedida al Ejecutivo de la Unión por decreto de 4 de Junio de 1887, 

he tenido á bien expedir el siguiente 
 

CODIGO DE COMERCIO 
 

LIBRO PRIMERO 
 

TITULO PRELIMINAR 
 
Artículo 1o.- Los actos comerciales sólo se regirán por lo dispuesto en este Código y las demás leyes 

mercantiles aplicables. 
 
Artículo 2o.- A falta de disposiciones de este ordenamiento y las demás leyes mercantiles, serán 

aplicables a los actos de comercio las del derecho común contenidas en el Código Civil aplicable en 
materia federal. 

 

TITULO PRIMERO 
De los Comerciantes 

 
Artículo 3o.- Se reputan en derecho comerciantes: 
 
I.- Las personas que teniendo capacidad legal para ejercer el comercio, hacen de él su ocupación 

ordinaria; 
 
II.- Las sociedades constituidas con arreglo a las leyes mercantiles; 
 
III.- Las sociedades extranjeras o las agencias y sucursales de éstas, que dentro del territorio nacional 

ejerzan actos de comercio. 
 
Artículo 4o.- Las personas que accidentalmente, con o sin establecimiento fijo, hagan alguna 

operación de comercio, aunque no son en derecho comerciantes, quedan sin embargo, sujetas por ella a 
las leyes mercantiles. Por tanto, los labradores y fabricantes, y en general todos los que tienen 
planteados almacén o tienda en alguna población para el expendio de los frutos de su finca, o de los 
productos ya elaborados de su industria, o trabajo, sin hacerles alteración al expenderlos, serán 
considerados comerciantes en cuanto concierne a sus almacenes o tiendas. 

 



 

 

 
 

 

  

Artículo 5o.- Toda persona que, según las leyes comunes, es hábil para contratar y obligarse, y a 
quien las mismas leyes no prohíben expresamente la profesión del comercio, tiene capacidad legal para 
ejercerlo. 

 
Artículo 6o.- (Se deroga). 
 
Artículo 6 bis. Los comerciantes deberán realizar su actividad de acuerdo a los usos honestos en 

materia industrial o comercial, por lo que se abstendrán de realizar actos de competencia desleal que: 
 
I.- Creen confusión, por cualquier medio que sea, respecto del establecimiento, los productos o la 

actividad industrial o comercial, de otro comerciante; 
 
II.- Desacrediten, mediante aseveraciones falsas, el establecimiento, los productos o la actividad 

industrial o comercial, de cualquier otro comerciante; 
 
III.- Induzcan al público a error sobre la naturaleza, el modo de fabricación, las características, la 

aptitud en el empleo o la cantidad de los productos, o 
 
IV.- Se encuentren previstos en otras leyes. 
 
Las acciones civiles producto de actos de competencia desleal, sólo podrán iniciarse cuando se haya 

obtenido un pronunciamiento firme en la vía administrativa, si ésta es aplicable. 
 
Artículo 7o.- (Se deroga). 
 
Artículo 8o.- (Se deroga). 
 
Artículo 9o.- Tanto el hombre como la mujer casados comerciantes, pueden hipotecar sus bienes 

raíces para seguridad de sus obligaciones mercantiles y comparecer en juicio sin necesidad de licencia 
del otro cónyuge, cuando el matrimonio se rija por el régimen de separación de bienes. 

 
En el régimen Social Conyugal, ni el hombre ni la mujer comerciantes, podrán hipotecar ni gravar los 

bienes de la sociedad, ni los suyos propios cuyos frutos o productos correspondan a la sociedad, sin 
licencia del otro cónyuge. 

 
Artículo 10.- (Se deroga). 
 
Artículo 11.- (Se deroga). 
 
Artículo 12.- No pueden ejercer el comercio: 
 
I.- Los corredores; 
 
II.- Los quebrados que no hayan sido rehabilitados; 
 
III.- Los que por sentencia ejecutoriada hayan sido condenados por delitos contra la propiedad, 

incluyendo en éstos la falsedad, el peculado, el cohecho y la concusión. 
 
La limitación a que se 

In [12]:
# Parse the text content
parsed_data = parse_text(decoded_data['TEXTO VIGENTE'])
#print(parsed_data)

In [13]:
parsed_data['CODIGO DE COMERCIO'].keys()

dict_keys(['LIBRO PRIMERO', 'LIBRO SEGUNDO', 'LIBRO TERCERO', 'LIBRO CUARTO', 'LIBRO QUINTO'])

In [14]:
parsed_data['CODIGO DE COMERCIO'].keys()

dict_keys(['LIBRO PRIMERO', 'LIBRO SEGUNDO', 'LIBRO TERCERO', 'LIBRO CUARTO', 'LIBRO QUINTO'])

In [15]:
#print(parsed_data['CODIGO DE COMERCIO']['LIBRO PRIMERO'])

In [16]:
parsed_libros = parse_libros(parsed_data['CODIGO DE COMERCIO'])

In [17]:
#print(parsed_libros)

# Instruct-tuning Dataset Preparation

{"context": "TITULO PRELIMINAR /n/nArtículo 1o.- Los actos comerciales sólo se regirán por lo dispuesto en este Código y las demás leyes /n/nmercantiles aplicables. /n/nArtículo 2o.- A falta de disposiciones de este ordenamiento y las demás leyes mercantiles, serán /n/naplicables a los actos de comercio las del derecho común contenidas en el Código Civil aplicable en /nmateria federal. ", 
question": "¿Cuál es el alcance del Código Civil Federal?",
"answer": "Según el Artículo 1o, las disposiciones del Código Civil Federal rigen en toda la República Mexicana en asuntos del orden federal."},
{"context": "CAPITULO I /n/nDe la Hipoteca en General /n/n/n/n/nArtículo 2893.- La hipoteca es una garantía real constituida sobre bienes que no se entregan al /nacreedor, y que da derecho a éste, en caso de incumplimiento de la obligación garantizada, a ser pagado /ncon el valor de los bienes, en el grado de preferencia establecido por la ley. /n/n/nArtículo 2894.- Los bienes hipotecados quedan sujetos al gravamen impuesto, aunque pasen a poder /n/nde tercero./n/n/nArtículo 2895.- La hipoteca sólo puede recaer sobre bienes especialmente determinados. /n/n/nArtículo 2896.- La hipoteca se extiende aunque no se exprese: ",
"question": "De acuerdo a las leyes mexicanas, ¿qué es una hipoteca?",
"answer": "De acuerdo al Artículo 2893, la hipoteca es una garantía real constituida sobre bienes que no se entregan al acreedor, y que da derecho a éste, en caso de incumplimiento de la obligación garantizada, a ser pagado con el valor de los bienes."}